# Cleaning & Compile Data Kesehatan Berdasarkan Persentase

#### By Aditya Hanif

In [1]:
import pandas as pd
import os
import glob

In [2]:
abc = glob.glob("/content/*.xlsx")
abc

['/content/Persentase Keluarga dengan Akses Sanitasi Layak (Jamban Sehat).xlsx',
 '/content/Persentase Desa yang Stop Buang Air Besar Sembarangan (BABS).xlsx',
 '/content/Persentase Sarana Air Minum yang Diambil Sampel.xlsx',
 '/content/Persentase Penderita Hipertensi yang Mendapat Pelayanan Kesehatan.xlsx',
 '/content/Persentase Desa atau Kelurahan Universal Child Immunization.xlsx',
 '/content/Persentase Ibu Nifas yang Mendapatkan VIT A.xlsx',
 '/content/Persentase Rumah Tangga Berperilaku Hidup Bersih dan Sehat (BER-PHBS).xlsx',
 '/content/Persentase Penderita Diabetes Melitus yang Mendapat Pelayanan Kesehatan Sesuai Standar.xlsx',
 '/content/Persentase Penduduk yang Mempunyai Keluhan Kesehatan.xlsx',
 '/content/Persentase Orang Dengan Gangguan Jiwa (ODGJ) Berat yang Mendapat Pelayanan Kesehatan.xlsx',
 '/content/Persentase Bayi Berat Badan Lahir Rendah (BBLR).xlsx',
 '/content/Persentase Pemberian Air Susu Ibu (ASI) Eksklusif pada Bayi Kurang Dari 6 Bulan.xlsx',
 '/content/Persenta

## Cleaning & Gabung Ke Dalam Satu Dataframe

In [3]:
# Folder tempat file-file Excel disimpan
folder_path = "/content/"  # ganti sesuai folder Anda

# List untuk menampung dataframe tiap file
all_data = []

for file in os.listdir(folder_path):
    if file.endswith(".xlsx"):
        file_path = os.path.join(folder_path, file)

        # Baca data
        df = pd.read_excel(file_path)

        # Mapping nama kolom
        # bila diawali jumlah_xxx dan kategori_xxx akan diubah jadi Jumlah dan Kategori
        new_columns = {}
        for col in df.columns:
            if col.startswith("persentase"):
                new_columns[col] = "Persentase"
            elif col.startswith("kategori"):
                new_columns[col] = "Kategori"
            elif col.startswith("jenis_"):
                new_columns[col] = "Kategori"
            elif col != "Jenis":
                new_columns[col] = col.replace("_", " ").title()

        df = df.rename(columns=new_columns)


        # Ubah semua value teks jadi kapital di awal kata
        for col in df.select_dtypes(include=["object"]).columns:
            df[col] = df[col].astype(str).str.title()

        # Tambahkan kolom 'Jenis' sesuai nama file (tanpa .xlsx)
        jenis_value = os.path.splitext(file)[0]
        df["Jenis"] = jenis_value

        # Masukkan ke list
        all_data.append(df)

# Gabungkan semua dataframe
final_df = pd.concat(all_data, ignore_index=True)
final_df.head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Persentase,Satuan,Tahun,Jenis
0,1,32,Jawa Barat,3201,Kabupaten Bogor,6.85,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
1,2,32,Jawa Barat,3202,Kabupaten Sukabumi,47.29,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
2,3,32,Jawa Barat,3203,Kabupaten Cianjur,38.96,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
3,4,32,Jawa Barat,3204,Kabupaten Bandung,57.97,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
4,5,32,Jawa Barat,3205,Kabupaten Garut,58.15,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...


## Cek Tipe Data

In [4]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3078 entries, 0 to 3077
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   3078 non-null   int64 
 1   Kode Provinsi        3078 non-null   int64 
 2   Nama Provinsi        3078 non-null   object
 3   Kode Kabupaten Kota  3078 non-null   int64 
 4   Nama Kabupaten Kota  3078 non-null   object
 5   Persentase           3078 non-null   object
 6   Satuan               3078 non-null   object
 7   Tahun                3078 non-null   int64 
 8   Jenis                3078 non-null   object
dtypes: int64(4), object(5)
memory usage: 216.6+ KB


## Mengubah Tipe Data Persentase Ke Float

In [5]:
final_df[final_df["Persentase"].astype(float)].info()

ValueError: could not convert string to float: '88,77'

##### Ternyata ada data yang menggunakan tanda koma, standar seharusnya tanda titik untuk angka desimal

## Masking & Cek Data Dengan Pemisah Tanda Koma

In [6]:
final_df[final_df['Persentase'].astype(str).str.contains(',')].info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 270 to 2617
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   50 non-null     int64 
 1   Kode Provinsi        50 non-null     int64 
 2   Nama Provinsi        50 non-null     object
 3   Kode Kabupaten Kota  50 non-null     int64 
 4   Nama Kabupaten Kota  50 non-null     object
 5   Persentase           50 non-null     object
 6   Satuan               50 non-null     object
 7   Tahun                50 non-null     int64 
 8   Jenis                50 non-null     object
dtypes: int64(4), object(5)
memory usage: 3.9+ KB


## Memisahkan & Memeriksa DF Yang Mengandung Titik dan Koma

In [7]:
df1_titik = final_df[~final_df['Persentase'].astype(str).str.contains(',')]
df1_koma = final_df[final_df['Persentase'].astype(str).str.contains(',')]

In [8]:
df1_titik.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3028 entries, 0 to 3077
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   3028 non-null   int64 
 1   Kode Provinsi        3028 non-null   int64 
 2   Nama Provinsi        3028 non-null   object
 3   Kode Kabupaten Kota  3028 non-null   int64 
 4   Nama Kabupaten Kota  3028 non-null   object
 5   Persentase           3028 non-null   object
 6   Satuan               3028 non-null   object
 7   Tahun                3028 non-null   int64 
 8   Jenis                3028 non-null   object
dtypes: int64(4), object(5)
memory usage: 236.6+ KB


In [9]:
df1_koma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 270 to 2617
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   50 non-null     int64 
 1   Kode Provinsi        50 non-null     int64 
 2   Nama Provinsi        50 non-null     object
 3   Kode Kabupaten Kota  50 non-null     int64 
 4   Nama Kabupaten Kota  50 non-null     object
 5   Persentase           50 non-null     object
 6   Satuan               50 non-null     object
 7   Tahun                50 non-null     int64 
 8   Jenis                50 non-null     object
dtypes: int64(4), object(5)
memory usage: 3.9+ KB


## Mengubah Koma Menjadi Titik dan Mengubah Ke Tipe Float

In [10]:
df1_koma["Persentase"] = df1_koma["Persentase"].str.replace(",", ".").astype(float)
df1_koma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 270 to 2617
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   50 non-null     int64  
 1   Kode Provinsi        50 non-null     int64  
 2   Nama Provinsi        50 non-null     object 
 3   Kode Kabupaten Kota  50 non-null     int64  
 4   Nama Kabupaten Kota  50 non-null     object 
 5   Persentase           50 non-null     float64
 6   Satuan               50 non-null     object 
 7   Tahun                50 non-null     int64  
 8   Jenis                50 non-null     object 
dtypes: float64(1), int64(4), object(4)
memory usage: 3.9+ KB


/tmp/ipython-input-725629929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_koma["Persentase"] = df1_koma["Persentase"].str.replace(",", ".").astype(float)


## Mengubah Ke Tipe Float

In [11]:
df1_titik["Persentase"] = df1_titik["Persentase"].astype(float)
df1_titik.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3028 entries, 0 to 3077
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   3028 non-null   int64  
 1   Kode Provinsi        3028 non-null   int64  
 2   Nama Provinsi        3028 non-null   object 
 3   Kode Kabupaten Kota  3028 non-null   int64  
 4   Nama Kabupaten Kota  3028 non-null   object 
 5   Persentase           3028 non-null   float64
 6   Satuan               3028 non-null   object 
 7   Tahun                3028 non-null   int64  
 8   Jenis                3028 non-null   object 
dtypes: float64(1), int64(4), object(4)
memory usage: 236.6+ KB


/tmp/ipython-input-2582075624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_titik["Persentase"] = df1_titik["Persentase"].astype(float)


## Menggabungkan Kembali DF yang Dipisah

In [12]:
df1 = pd.concat([df1_titik, df1_koma], ignore_index=True)
df1.head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Persentase,Satuan,Tahun,Jenis
0,1,32,Jawa Barat,3201,Kabupaten Bogor,6.85,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
1,2,32,Jawa Barat,3202,Kabupaten Sukabumi,47.29,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
2,3,32,Jawa Barat,3203,Kabupaten Cianjur,38.96,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
3,4,32,Jawa Barat,3204,Kabupaten Bandung,57.97,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...
4,5,32,Jawa Barat,3205,Kabupaten Garut,58.15,Persen,2014,Persentase Keluarga dengan Akses Sanitasi Laya...


## Sortir DF Berdasarkan Jenis, Tahun, dan Kode Kab/Kota

In [13]:
df1_final = df1.sort_values(by=["Jenis", "Tahun", "Kode Kabupaten Kota"])
df1_final.to_excel("df1.xlsx", index=False)

## Cleaning & Gabung Data 2 Ke Dalam Satu Dataframe

In [27]:
abcd = glob.glob("/content/jns/*.xlsx")
abcd

['/content/jns/Persentase Pelayanan Kesehatan Pada Ibu Hamil [Berdasarkan Jenis Kunjungan].xlsx',
 '/content/jns/Persentase Kunjungan Neonatal [Berdasarkan Jenis Kunjungan].xlsx']

In [28]:
# Folder tempat file-file Excel disimpan
lokasi = "/content/jns"  # ganti sesuai folder Anda

# List untuk menampung dataframe tiap file
datas = []

for file in os.listdir(lokasi):
    if file.endswith(".xlsx"):
        lok_file = os.path.join(lokasi, file)

        # Baca data
        df2 = pd.read_excel(lok_file)

        # Mapping nama kolom
        # bila diawali jumlah_xxx dan kategori_xxx akan diubah jadi Jumlah dan Kategori
        new_columns = {}
        for col in df2.columns:
            if col.startswith("persentase"):
                new_columns[col] = "Persentase"
            elif col.startswith("kategori"):
                new_columns[col] = "Kategori"
            elif col.startswith("jenis_kunjungan"):
                new_columns[col] = "Kategori"
            elif col != "Jenis":
                new_columns[col] = col.replace("_", " ").title()

        df2 = df2.rename(columns=new_columns)


        # Ubah semua value teks jadi kapital di awal kata
        for col in df2.select_dtypes(include=["object"]).columns:
            df2[col] = df2[col].astype(str).str.title()

        # Tambahkan kolom 'Jenis' sesuai nama file (tanpa .xlsx)
        jenis_value = os.path.splitext(file)[0]
        df2["Jenis"] = jenis_value

        # Masukkan ke list
        datas.append(df2)

# Gabungkan semua dataframe
final_df2 = pd.concat(datas, ignore_index=True)
final_df2.head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
0,1,32,Jawa Barat,3201,Kabupaten Bogor,K1,101,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
1,2,32,Jawa Barat,3201,Kabupaten Bogor,K4,91.8,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
2,3,32,Jawa Barat,3202,Kabupaten Sukabumi,K1,107.2,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
3,4,32,Jawa Barat,3202,Kabupaten Sukabumi,K4,94.8,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
4,5,32,Jawa Barat,3203,Kabupaten Cianjur,K1,107.8,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...


## Cek Tipe Data Ke-2

In [29]:
final_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   729 non-null    int64 
 1   Kode Provinsi        729 non-null    int64 
 2   Nama Provinsi        729 non-null    object
 3   Kode Kabupaten Kota  729 non-null    int64 
 4   Nama Kabupaten Kota  729 non-null    object
 5   Kategori             729 non-null    object
 6   Persentase           729 non-null    object
 7   Satuan               729 non-null    object
 8   Tahun                729 non-null    int64 
 9   Jenis                729 non-null    object
dtypes: int64(4), object(6)
memory usage: 57.1+ KB


## Mengubah Tipe Data Persentase Ke Float

In [30]:
final_df2[final_df2["Persentase"].astype(float)].info()

ValueError: could not convert string to float: '101,53'

## Masking & Cek Data Dengan Pemisah Tanda Koma

In [31]:
final_df2[final_df2['Persentase'].astype(str).str.contains(',')].info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 324 to 403
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   80 non-null     int64 
 1   Kode Provinsi        80 non-null     int64 
 2   Nama Provinsi        80 non-null     object
 3   Kode Kabupaten Kota  80 non-null     int64 
 4   Nama Kabupaten Kota  80 non-null     object
 5   Kategori             80 non-null     object
 6   Persentase           80 non-null     object
 7   Satuan               80 non-null     object
 8   Tahun                80 non-null     int64 
 9   Jenis                80 non-null     object
dtypes: int64(4), object(6)
memory usage: 6.9+ KB


## Memisahkan dan Memeriksa Data yang Mengandung Titik dan Koma

In [32]:
df2_titik = final_df2[~final_df2['Persentase'].astype(str).str.contains(',')]
df2_koma = final_df2[final_df2['Persentase'].astype(str).str.contains(',')]

In [33]:
df2_titik.info()
df2_koma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 649 entries, 0 to 728
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   649 non-null    int64 
 1   Kode Provinsi        649 non-null    int64 
 2   Nama Provinsi        649 non-null    object
 3   Kode Kabupaten Kota  649 non-null    int64 
 4   Nama Kabupaten Kota  649 non-null    object
 5   Kategori             649 non-null    object
 6   Persentase           649 non-null    object
 7   Satuan               649 non-null    object
 8   Tahun                649 non-null    int64 
 9   Jenis                649 non-null    object
dtypes: int64(4), object(6)
memory usage: 55.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 324 to 403
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Id                   80 non-null     int64 
 1   Kode Pro

## Mengubah Koma ke Titik dan Mengubah Tipe Data Menjadi Float

In [34]:
df2_koma["Persentase"] = df2_koma["Persentase"].str.replace(",", ".").astype(float)
df2_koma.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 324 to 403
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   80 non-null     int64  
 1   Kode Provinsi        80 non-null     int64  
 2   Nama Provinsi        80 non-null     object 
 3   Kode Kabupaten Kota  80 non-null     int64  
 4   Nama Kabupaten Kota  80 non-null     object 
 5   Kategori             80 non-null     object 
 6   Persentase           80 non-null     float64
 7   Satuan               80 non-null     object 
 8   Tahun                80 non-null     int64  
 9   Jenis                80 non-null     object 
dtypes: float64(1), int64(4), object(5)
memory usage: 6.9+ KB


/tmp/ipython-input-1814803313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_koma["Persentase"] = df2_koma["Persentase"].str.replace(",", ".").astype(float)


## Mengubah Tipe Data Menjadi Float

In [35]:
df2_titik["Persentase"] = df2_titik["Persentase"].astype(float)
df2_titik.info()

<class 'pandas.core.frame.DataFrame'>
Index: 649 entries, 0 to 728
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   649 non-null    int64  
 1   Kode Provinsi        649 non-null    int64  
 2   Nama Provinsi        649 non-null    object 
 3   Kode Kabupaten Kota  649 non-null    int64  
 4   Nama Kabupaten Kota  649 non-null    object 
 5   Kategori             649 non-null    object 
 6   Persentase           649 non-null    float64
 7   Satuan               649 non-null    object 
 8   Tahun                649 non-null    int64  
 9   Jenis                649 non-null    object 
dtypes: float64(1), int64(4), object(5)
memory usage: 55.8+ KB


/tmp/ipython-input-4252905916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_titik["Persentase"] = df2_titik["Persentase"].astype(float)


## Menggabungkan Data yang Telah Dipisah

In [36]:
df2 = pd.concat([df2_titik, df2_koma], ignore_index=True)
df2.head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
0,1,32,Jawa Barat,3201,Kabupaten Bogor,K1,101.0,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
1,2,32,Jawa Barat,3201,Kabupaten Bogor,K4,91.8,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
2,3,32,Jawa Barat,3202,Kabupaten Sukabumi,K1,107.2,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
3,4,32,Jawa Barat,3202,Kabupaten Sukabumi,K4,94.8,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...
4,5,32,Jawa Barat,3203,Kabupaten Cianjur,K1,107.8,Persen,2019,Persentase Pelayanan Kesehatan Pada Ibu Hamil ...


## Sortir DF Berdasarkan Jenis, Tahun, Kode Kab/Kota, dan Kategori

In [37]:
df2_final = df2.sort_values(by=["Jenis", "Tahun", "Kode Kabupaten Kota", "Kategori"])
df2_final.head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
325,1,32,Jawa Barat,3201,Kabupaten Bogor,1 Kali (Kn 1),100.5,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
326,2,32,Jawa Barat,3201,Kabupaten Bogor,3 Kali (Kn Lengkap),98.6,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
327,3,32,Jawa Barat,3202,Kabupaten Sukabumi,1 Kali (Kn 1),106.2,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
328,4,32,Jawa Barat,3202,Kabupaten Sukabumi,3 Kali (Kn Lengkap),102.9,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
329,5,32,Jawa Barat,3203,Kabupaten Cianjur,1 Kali (Kn 1),104.9,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...


## Cek Data yang Huruf Kapital Tidak Sesuai

In [43]:
df2_final[df2_final['Kategori'].astype(str).str.contains('Kn')].head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
325,1,32,Jawa Barat,3201,Kabupaten Bogor,1 Kali (Kn 1),100.5,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
326,2,32,Jawa Barat,3201,Kabupaten Bogor,3 Kali (Kn Lengkap),98.6,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
327,3,32,Jawa Barat,3202,Kabupaten Sukabumi,1 Kali (Kn 1),106.2,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
328,4,32,Jawa Barat,3202,Kabupaten Sukabumi,3 Kali (Kn Lengkap),102.9,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
329,5,32,Jawa Barat,3203,Kabupaten Cianjur,1 Kali (Kn 1),104.9,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...


## Menyesuaikan Huruf Kapital

In [44]:
df2_final['Kategori'] = df2_final['Kategori'].str.replace('Kn', 'KN')
df2_final.head(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
325,1,32,Jawa Barat,3201,Kabupaten Bogor,1 Kali (KN 1),100.5,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
326,2,32,Jawa Barat,3201,Kabupaten Bogor,3 Kali (KN Lengkap),98.6,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
327,3,32,Jawa Barat,3202,Kabupaten Sukabumi,1 Kali (KN 1),106.2,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
328,4,32,Jawa Barat,3202,Kabupaten Sukabumi,3 Kali (KN Lengkap),102.9,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...
329,5,32,Jawa Barat,3203,Kabupaten Cianjur,1 Kali (KN 1),104.9,Persen,2019,Persentase Kunjungan Neonatal [Berdasarkan Jen...


## Menggabungkan Data Pertama dan Kedua

In [46]:
df_all = pd.concat([df2_final, df1_final], ignore_index=True)
df_all.tail(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
3802,158,32,Jawa Barat,3275,Kota Bekasi,NaN,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3803,159,32,Jawa Barat,3276,Kota Depok,NaN,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3804,160,32,Jawa Barat,3277,Kota Cimahi,NaN,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3805,161,32,Jawa Barat,3278,Kota Tasikmalaya,NaN,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3806,162,32,Jawa Barat,3279,Kota Banjar,NaN,85.71,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel


## Mengisi Nan Value Kolom Kategori

In [47]:
df_all["Kategori"] = df_all["Kategori"].fillna("-")
df_all.tail(5)

,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis
3802,158,32,Jawa Barat,3275,Kota Bekasi,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3803,159,32,Jawa Barat,3276,Kota Depok,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3804,160,32,Jawa Barat,3277,Kota Cimahi,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3805,161,32,Jawa Barat,3278,Kota Tasikmalaya,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel
3806,162,32,Jawa Barat,3279,Kota Banjar,-,85.71,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel


## Menambahkan Kolom Region Google

In [48]:
name_map = {
    'Kabupaten Bandung': 'Bandung Regency',
    'Kabupaten Bandung Barat': 'West Bandung Regency',
    'Kabupaten Bekasi': 'Bekasi Regency',
    'Kabupaten Bogor': 'Bogor Regency',
    'Kabupaten Ciamis': 'Ciamis Regency',
    'Kabupaten Cianjur': 'Cianjur Regency',
    'Kabupaten Cirebon': 'Cirebon Regency',
    'Kabupaten Garut': 'Garut Regency',
    'Kabupaten Indramayu': 'Indramayu Regency',
    'Kabupaten Karawang': 'Karawang Regency',
    'Kabupaten Kuningan': 'Kuningan Regency',
    'Kabupaten Majalengka': 'Majalengka Regency',
    'Kabupaten Pangandaran': 'Pangandaran Regency',
    'Kabupaten Purwakarta': 'Purwakarta Regency',
    'Kabupaten Subang': 'Subang Regency',
    'Kabupaten Sukabumi': 'Sukabumi',
    'Kabupaten Sumedang': 'Sumedang Regency',
    'Kabupaten Tasikmalaya': 'Tasikmalaya Regency',
    'Kota Bandung': 'Bandung City',
    'Kota Banjar': 'Banjar City',
    'Kota Bekasi': 'Bekasi City',
    'Kota Bogor': 'Bogor City',
    'Kota Cimahi': 'Cimahi City',
    'Kota Cirebon': 'Cirebon City',
    'Kota Depok': 'Depok City',
    'Kota Sukabumi': 'Sukabumi City',
    'Kota Tasikmalaya': 'Tasikmalaya City'
}

# MEMBUAT KOLOM BARU DENGAN NAMA SESUAI GOOGLE
# Mengganti "Nama Kabupaten Kota" dengan nama kolom
df_all['Region Google'] = df_all['Nama Kabupaten Kota'].map(name_map).fillna(df_all['Nama Kabupaten Kota'])

print("✅ Selesai! Kolom Region_Google untuk Looker Studio sudah ditambahkan.")
df_all.tail(5)

✅ Selesai! Kolom Region_Google untuk Looker Studio sudah ditambahkan.


,Id,Kode Provinsi,Nama Provinsi,Kode Kabupaten Kota,Nama Kabupaten Kota,Kategori,Persentase,Satuan,Tahun,Jenis,Region Google
3802,158,32,Jawa Barat,3275,Kota Bekasi,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel,Bekasi City
3803,159,32,Jawa Barat,3276,Kota Depok,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel,Depok City
3804,160,32,Jawa Barat,3277,Kota Cimahi,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel,Cimahi City
3805,161,32,Jawa Barat,3278,Kota Tasikmalaya,-,100.00,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel,Tasikmalaya City
3806,162,32,Jawa Barat,3279,Kota Banjar,-,85.71,Persen,2024,Persentase Sarana Air Minum yang Diambil Sampel,Banjar City


## Export Data Yang Telah Digabung

In [49]:
df_all.to_excel("Data Persentase (Utuh).xlsx", index=False)

In [55]:
df_all.describe()

,Id,Kode Provinsi,Kode Kabupaten Kota,Persentase,Tahun
count,3807.000000,3807.0,3807.000000,3807.000000,3807.000000
mean,113.382979,32.0,3231.333333,68.718360,2020.936170
std,82.005898,0.0,31.205951,37.131768,2.334793
min,1.000000,32.0,3201.000000,0.000000,2014.000000
25%,51.000000,32.0,3207.000000,37.025000,2019.000000
50%,101.000000,32.0,3214.000000,82.300000,2021.000000
75%,152.000000,32.0,3273.000000,98.500000,2023.000000
max,405.000000,32.0,3279.000000,264.980000,2024.000000


## Normalisasi Data Persentase > 100 Menjadi 100

In [56]:
df_all.loc[df_all['Persentase'] > 100, 'Persentase'] = 100

In [58]:
df_all.describe()

,Id,Kode Provinsi,Kode Kabupaten Kota,Persentase,Tahun
count,3807.000000,3807.0,3807.000000,3807.000000,3807.000000
mean,113.382979,32.0,3231.333333,67.355666,2020.936170
std,82.005898,0.0,31.205951,35.301693,2.334793
min,1.000000,32.0,3201.000000,0.000000,2014.000000
25%,51.000000,32.0,3207.000000,37.025000,2019.000000
50%,101.000000,32.0,3214.000000,82.300000,2021.000000
75%,152.000000,32.0,3273.000000,98.500000,2023.000000
max,405.000000,32.0,3279.000000,100.000000,2024.000000


## Export Data

In [59]:
df_all.to_excel("Data Persentase (Adjusted).xlsx", index=False)